<a href="https://colab.research.google.com/github/jcestevezc/Machine-Learning-Techniques/blob/master/Laboratorio%207/NN_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://cursos.virtual.uniandes.edu.co/isis4219/wp-content/uploads/sites/162/2014/11/cropped-misisheader.png" ><br>
# Machine Learning Techniques - ISIS4219

Intersemestral 2020




## Nombre de la red

¿Que es una red XXXX?

### **1. Razones para utilizar la red**



### **2. Aplicaciones de la red**

### **3. Arquitectura de la red**

### **4. La red en Keras**

Keras no tiene una Echo State Network (ESN) implementada por defecto. Lo que sí se puede hacer es crear una celda personalizada para después utilizarla. A continuación se muestra una implementación encontrada en https://github.com/francesco-mannella/Echo-State-Networks/tree/master. Lo que hace es crear una celda personalizada y con ella construir las capas de una red neuronal recurrente (RNN). Al final, le agrega una última capa de salida densa que normalmente se le llama *readouts*. La implementación de la celda ESN la pueden encontrar en el siguiente link: https://github.com/francesco-mannella/Echo-State-Networks/blob/master/ESN.py

In [ ]:
# Init the ESN cell
cell = EchoStateRNNCell(units=num_units, 
                        activation=activation, 
                        decay=0.1, 
                        epsilon=1e-20,
                        alpha=0.5,
                        optimize=True,
                        optimize_vars=["rho", "decay", "alpha", "sw"],
                        seed=random_seed)

# Build the recurrent layer containing the ESN cell
recurrent_layer = keras.layers.RNN(cell, input_shape=(stime, num_inputs), 
                                   return_sequences=True, name="nn")
# Build the readout layer
output = keras.layers.Dense(num_outputs, name="readouts")
# initialize the adam optimizer for training
optimizer = keras.optimizers.Adam(learning_rate=0.01)

# put all together in a keras sequential model
model = keras.models.Sequential()
model.add(recurrent_layer)
model.add(output)

In [ ]:
model.compile(loss="mse", optimizer=optimizer)
model.summary()

### **5. Propiedades e hiperparametros**

Lo hiperparámetros para una ESN son los siguientes:

* units (int): Número de unidades de la celda RNN
* decay (float): Deterioro (decay) de la ODE de cada unidad. Default: 0.1.
* seed (int): Semilla para los números aleatorios. Default None.
* epsilon (float): Descuento al radio spectral de 1. Default: 1e-10.
* alpha (float): [0,1], la proporsión infinitesimal de expansión vs la rotación infinitesimal de un sistema dunámico definido por sus pesos internos
* sparseness (float): [0,1], Qué tan espacidos están los pesos de la matriz interna Default: 0.
* rho (float): Escala de los pesos internos
* sw (float): Escala de los pesos de entrada
* activation (callable): Función de activación a usar  Default: `tanh`.

Referencia: Tomado de https://github.com/francesco-mannella/Echo-State-Networks/blob/master/ESN.py#L39

### **6. Ejemplo practico - Demo**

Nota: para el desarrollo de este taller, tomar como referencia el siguiente ejemplo: [autoencoders](https://www.edureka.co/blog/autoencoders-tutorial/).